In [1]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [2]:
data = pd.read_excel("komentar/data_clean_from_dataset_new.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
print(len(y))
data.head()

4948


,Unnamed: 0,no,label,komentar
0,0,0,non spam,waduh mara ken ae mbak via ni kirim balikpapan...
1,1,1,spam,awass akun adu domba min tu org suka nyinyirin...
2,2,2,non spam,begini kalo ga biasa lipsing sekali lipsing ku...
3,3,3,non spam,nitijen mha bebar bkin mlu 2 ajh kliaan apa sd...
4,4,4,spam,maantaap banggeet kaak ssetelaah akuu pakee ge...


# Ekstraksi dan Seleksi Fitur

In [4]:
sf = cs.ChiSquare(alpha = 0.001)
sf.find_best_features(komentar, label)

In [13]:
#print("jumlah fitur terbaik :",len(sf.best_features),"dari",len(sf.fitur), "fitur")
#best_features = sf.best_features
#sf.data #untuk memunculkan semua data #ini disimpan ditabel bisa gk
#sf.alpha #untuk mengetahui alpha
sf.best_features #untuk melihat feature yg dipilih

['00',
 '0822',
 '0858',
 '0896',
 '100',
 '100ampuh',
 '10k',
 '10kg',
 '150',
 '15harii',
 '165',
 '1bulan',
 '1minggu',
 '1mingguu',
 '1x',
 '200',
 '25',
 '250',
 '265',
 '269',
 '2an',
 '2minggu',
 '2mingguu',
 '300',
 '350',
 '40ec',
 '50',
 '500',
 '50k',
 '55kg',
 '5kg',
 '5kgg',
 '6281646953',
 '62838',
 '6283829675',
 '6491',
 '64gb',
 '6kg',
 '6s',
 '7894',
 '7kg',
 '8559683975',
 '8kg',
 '900',
 'a39',
 'a5',
 'a57',
 'aamiin',
 'acara',
 'adaa',
 'add',
 'adil',
 'ae',
 'agen',
 'aj',
 'aja',
 'ajaa',
 'ajaib',
 'ajaibb',
 'ak',
 'akhir',
 'akkuu',
 'aku',
 'akuu',
 'al',
 'alaamii',
 'alami',
 'alat',
 'alhamdulillah',
 'allah',
 'alloh',
 'ama',
 'aman',
 'amann',
 'amazing',
 'amiin',
 'amin',
 'ampuh',
 'ampuhh',
 'apa',
 'apaa',
 'apaalagii',
 'aplikasi',
 'app',
 'appss',
 'apss',
 'aq',
 'arisan',
 'arti',
 'artis',
 'artiss',
 'asi',
 'asli',
 'ataass',
 'atas',
 'atass',
 'atauu',
 'att',
 'awal',
 'awalnyaa',
 'awat',
 'ayo',
 'ayoo',
 'ayu',
 'ayuu',
 'baanyakk'

# Pemodelan

## Pemodelan TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary = best_features)
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")

# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(4948, 845)


## Pemodelan MNB 

In [8]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Pelatihan dan Validasi Model

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.001
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
akurasi = list()

ix = 0
for alpha in alpha_list:
    clf.set_params(alpha=alpha)
    temp_akurasi = list()
    
    for train_index, test_index in skf.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        #latih model
        clf.fit(X_train,y_train)
        
        #prediksi
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
        
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)

    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
        print("")

        pass

ditt = {
    "alpha":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................


,alpha,akurasi
0,0.001,0.948484
1,0.002,0.948687
2,0.003,0.949293
3,0.004,0.949494
4,0.005,0.949494
5,0.006,0.949494
6,0.007,0.949494
7,0.008,0.949292
8,0.009,0.949292
9,0.010,0.949292


In [10]:
print(alpha_list[index])
print(akurasi[index])

0.528
0.9509060336946584


In [11]:
print(alpha_list[index])
print(akurasi[index])

0.528
0.9509060336946584


# Menyimpan Model

In [12]:
clf.set_params(alpha=alpha_list[index])
clf.fit(X, y)

dump(clf, "model/mnb.m")

['model/mnb.m']

In [15]:
# p = clf.predict(X)

In [30]:
# ditt = {
#     "Prediksi":p,
#     "label":label,
#     "komentar":komentar
# }
# # index = akurasi.index(max(akurasi))
# akk = pd.DataFrame.from_dict(ditt)

In [31]:
# akk.to_excel("coba_.xlsx")